# Bài toán Người Du Lịch (TSP) dùng Genetic Algorithm
Sử dụng dữ liệu từ `results.csv`

In [ ]:
import pandas as pd
import numpy as np
import random
import math
from haversine import haversine, Unit
import matplotlib.pyplot as plt


In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv('results.csv')
df.head()


In [ ]:
# Tính ma trận khoảng cách giữa các địa điểm
locations = list(zip(df['Latitude'], df['Longitude']))
num_locations = len(locations)

def compute_distance_matrix(locations):
    dist_matrix = np.zeros((num_locations, num_locations))
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                dist_matrix[i][j] = haversine(locations[i], locations[j])
    return dist_matrix

distance_matrix = compute_distance_matrix(locations)


In [ ]:
# Hàm tính tổng khoảng cách của một hành trình
def total_distance(tour, distance_matrix):
    dist = 0
    for i in range(len(tour)):
        dist += distance_matrix[tour[i]][tour[(i + 1) % len(tour)]]
    return dist


In [ ]:
# Khởi tạo quần thể ban đầu
def create_initial_population(pop_size, num_locations):
    population = []
    for _ in range(pop_size):
        tour = list(range(num_locations))
        random.shuffle(tour)
        population.append(tour)
    return population


In [ ]:
# Chọn lọc: tournament selection
def selection(population, distance_matrix, k=3):
    selected = random.sample(population, k)
    selected.sort(key=lambda x: total_distance(x, distance_matrix))
    return selected[0]


In [ ]:
# Lai ghép: Order Crossover (OX)
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end+1] = parent1[start:end+1]
    fill = [item for item in parent2 if item not in child]
    pointer = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[pointer]
            pointer += 1
    return child


In [ ]:
# Đột biến: hoán đổi vị trí 2 điểm
def mutate(tour, mutation_rate=0.01):
    for i in range(len(tour)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(tour) - 1)
            tour[i], tour[j] = tour[j], tour[i]
    return tour


In [ ]:
# GA chính
def genetic_algorithm(distance_matrix, pop_size=100, generations=500, mutation_rate=0.01):
    population = create_initial_population(pop_size, len(distance_matrix))
    best_distance = float('inf')
    best_solution = None
    history = []

    for gen in range(generations):
        new_population = []
        for _ in range(pop_size):
            p1 = selection(population, distance_matrix)
            p2 = selection(population, distance_matrix)
            child = crossover(p1, p2)
            child = mutate(child, mutation_rate)
            new_population.append(child)

        population = new_population
        gen_best = min(population, key=lambda x: total_distance(x, distance_matrix))
        dist = total_distance(gen_best, distance_matrix)
        history.append(dist)
        if dist < best_distance:
            best_distance = dist
            best_solution = gen_best

    return best_solution, best_distance, history


In [ ]:
# Chạy thuật toán
best_tour, best_dist, history = genetic_algorithm(distance_matrix)

print("Tổng quãng đường ngắn nhất:", round(best_dist, 2), "km")
print("Hành trình tốt nhất:", best_tour)

# Vẽ tiến trình tối ưu
plt.plot(history)
plt.xlabel("Thế hệ")
plt.ylabel("Tổng quãng đường")
plt.title("Tiến trình tối ưu GA")
plt.grid(True)
plt.show()
